In [ ]:
import pandas as pd
import numpy as np
import os

home = os.path.expanduser('~')
movies = pd.read_csv(home + '/Downloads/movies.csv')
ratings = pd.read_csv(home + '/Downloads/ratings.csv')
movies

In [ ]:
ratings

In [ ]:
ratings = rating[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix

In [ ]:
rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

In [ ]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarirt(ratings_matrix_T, ratings_matrix_T)

item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
item_sim_df

In [ ]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

In [ ]:
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

In [ ]:
def predict_rating(ratings_arr, item_sim_arr):
    return ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])

In [ ]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)
ratings_pred_matrix

In [ ]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
get_mse(ratings_pred, ratings_matrix.values)

In [ ]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    pred = np.zeros(ratings_arr.shape)
    
    for col in range(ratings_arr.shape[1]):
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
            
    return pred

In [ ]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print(get_mse(ratings_pred, ratings_matrix.values))

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)
ratings_pred_matrix

In [ ]:
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    already_seen = user_rating[user_rating>0].index.tolist()
    movies_list = ratings_matrix.columns.tolist()
    
    return [movie for movie in movies_list if movie not in already_seen]

In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    return pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]

In [ ]:
unseen_list = get_unseen_movies(ratings_matrix, 9)

In [ ]:
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

In [ ]:
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                            columns=['pred_score'])
recomm_movies